**Алгоритм обработки заказов в Озон** 

Алгоритм подготавливает файлы для формирования дашбордов в Tableau, Yandex Datalens

Все конфиденциальные данные изменены. 

Дашборд в Tableau: <https://public.tableau.com/app/profile/alexey.chernov/viz/cecrececececec/sheet6>

Дашборд в Yandex Datalens: <https://datalens.yandex/m37eg7fmkk10c>

In [1]:
# Импортируем нужные библиотеки, запустим подключения и прочитаем датафреймы
import pandas as pd
import datetime
pd.set_option('display.float_format', '{:.2f}'.format)
fbs = pd.read_csv('fbs.csv', sep=';')
fbo = pd.read_csv('fbo.csv', sep=';')
sku = pd.read_excel('sku.xlsx')
ozon_past_months = pd.read_excel('ozon_past_months.xlsx')
commission_ozon = pd.read_excel('Комиссии.xlsx')

In [2]:
# Добавим столбец со складом
fbs['Склад'] = 'FBS'
fbo['Склад'] = 'FBO'

# Удалим время в столбце
def delete_time(row):
    time = row['Принят в обработку']
    return time.split(" ")[0]
fbs['Принят в обработку'] = fbs.apply(delete_time, axis=1)
fbo['Принят в обработку'] = fbo.apply(delete_time, axis=1)

# Поменяем формат столбца "Принят в обработку" на дату
fbs['Принят в обработку'] = pd.to_datetime(fbs['Принят в обработку'], format='%Y-%m-%d')
#fbo['Принят в обработку'] = pd.to_datetime(fbo['Принят в обработку'], format='%d.%m.%Y')
fbo['Принят в обработку'] = pd.to_datetime(fbo['Принят в обработку'], format='%Y-%m-%d')

# Поменяем формат столбца "Принят в обработку" на дату
#fbs['Принят в обработку'] = pd.to_datetime(fbs['Принят в обработку']).dt.date
#fbo['Принят в обработку'] = pd.to_datetime(fbo['Принят в обработку']).dt.date

# Избавимся от отмененных заказов
fbs = fbs.query('Статус != "Отменен"')
fbo = fbo.query('Статус != "Отменен"')

# Избавимся от лишних столбцов
use = ['Принят в обработку', 'Сумма отправления', 'Наименование товара', 'Артикул', 'Итоговая стоимость товара', 'Количество', 'Склад']
use_2 = ['Артикул', 'Название линейки', 'Бренд', 'Номенклатура', 'Себестоимость']
fbs = fbs[use]
fbo = fbo[use]
sku = sku[use_2]

# Создадим таблицу с комиссиями fbo, оставим нужные столбцы
commission_ozon_fbo = commission_ozon[[
    'Артикул', 
    'Размер комиссии, %',
    'Логистика Ozon, максимум, FBO',
    'Последняя миля, FBO'
]]
commission_ozon_fbo['Комиссия доставки'] = commission_ozon_fbo['Логистика Ozon, максимум, FBO'] + commission_ozon_fbo['Последняя миля, FBO']
commission_ozon_fbo = commission_ozon_fbo[[
    'Артикул',
    'Размер комиссии, %',
    'Комиссия доставки'
]]

# Создадим таблицу с комиссиями fbs, оставим нужные столбцы
commission_ozon_fbs = commission_ozon[[
    'Артикул', 
    'Размер комиссии, %',
    'Обработка отправления, максимум FBS',
    'Логистика Ozon (РФ), FBS',
    'Последняя миля, FBS'
]]
commission_ozon_fbs['Комиссия доставки'] = commission_ozon_fbs['Обработка отправления, максимум FBS'] + commission_ozon_fbs['Логистика Ozon (РФ), FBS'] + commission_ozon_fbs['Последняя миля, FBS'] 
commission_ozon_fbs = commission_ozon_fbs[[
    'Артикул',
    'Размер комиссии, %',
    'Комиссия доставки'
]]

# Сменим тип данных в Артикуле
fbs['Артикул'] = fbs['Артикул'].astype('int')
fbo['Артикул'] = fbo['Артикул'].astype('int')

# Добавим комиссии 
fbs = pd.merge(fbs, commission_ozon_fbs, how='left', on=('Артикул'))
fbo = pd.merge(fbo, commission_ozon_fbo, how='left', on=('Артикул'))

# Объединим датафреймы в ozon
ozon = fbo.append(fbs).rename(columns={"Итоговая стоимость товара": "Цена товара"})

# Добавим дополнительные поля по артикулу
ozon = pd.merge(ozon, sku, how='left', on=('Артикул'))
ozon = ozon.drop(columns='Наименование товара', axis=1)

# Выгрузим первые 5 строк
ozon.head()

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_11524/439965144.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commission_ozon_fbo['Комиссия доставки'] = commission_ozon_fbo['Логистика Ozon, максимум, FBO'] + commission_ozon_fbo['Последняя миля, FBO']
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_11524/439965144.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  commission_ozon_fbs['Комиссия доставки'] = commission_ozon_fbs['Обработка отправления, максимум FBS'] + commission_

,Принят в обработку,Сумма отправления,Артикул,Цена товара,Количество,Склад,"Размер комиссии, %",Комиссия доставки,Название линейки,Бренд,Номенклатура,Себестоимость
0,2023-05-09,2620.00,28491,2620.00,1,FBO,10.15,184.10,Линейка 154,Бренд 3,Номенклатура 220,2044.94
1,2023-05-09,4030.00,12836,4030.00,1,FBO,10.15,262.65,Линейка 157,Бренд 3,Номенклатура 222,3202.77
2,2023-05-09,630.00,12200,630.00,1,FBO,10.15,74.65,Линейка 144,Бренд 11,Номенклатура 205,139.51
3,2023-05-09,1040.00,5298,1040.00,1,FBO,10.15,97.20,Линейка 144,Бренд 11,Номенклатура 206,251.45
4,2023-05-09,810.00,121531,810.00,1,FBO,10.15,84.55,Линейка 129,Бренд 11,Номенклатура 175,251.45


In [3]:
# Создадим столбец с подсчетом маржинальности
ozon['Маржинальность'] = ozon['Сумма отправления'] - (ozon['Размер комиссии, %']/100*ozon['Сумма отправления']) - (ozon['Комиссия доставки']*ozon['Количество']) - (ozon['Себестоимость']*ozon['Количество'])

In [4]:
# Сгруппируем данные, чтобы сэкономить количество строк
ozon = (
    ozon.groupby(['Принят в обработку', 'Артикул', 'Цена товара'])
    .agg({
        'Сумма отправления':'sum',
        'Количество':'sum',
        'Маржинальность':'sum',
        'Название линейки':'first',
        'Бренд':'first',
        'Номенклатура':'first'
    })
    .rename(columns={"Название линейки": "Наименование общее"})
    .reset_index()   
)

# Объединим с прошлыми месяцами
ozon = ozon.append(ozon_past_months).dropna()

# Выгрузим в эксель для табло
ozon.to_excel('ozon_tableau.xlsx', index=False)

# Выгрузим в csv для datalens
ozon.to_csv('ozon_datalens.csv', index=False)

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_11524/1616079094.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon = ozon.append(ozon_past_months).dropna()


,Принят в обработку,Артикул,Цена товара,Сумма отправления,Количество,Маржинальность,Наименование общее,Бренд,Номенклатура
0,2023-05-01,130,1730.00,1730.00,1,107.95,Линейка 126,Бренд 1,Номенклатура 170
1,2023-05-01,136,1730.00,1730.00,1,82.95,Линейка 126,Бренд 1,Номенклатура 170
2,2023-05-01,144,1730.00,3460.00,2,215.91,Линейка 126,Бренд 1,Номенклатура 170
3,2023-05-01,147,1730.00,3460.00,2,165.91,Линейка 126,Бренд 1,Номенклатура 170
4,2023-05-01,155,1580.00,1580.00,1,-26.82,Линейка 126,Бренд 1,Номенклатура 170
...,...,...,...,...,...,...,...,...,...
182717,2022-09-30,981086600,1150.00,1150.00,1,68.00,Линейка 149,Бренд 3,Номенклатура 212
182718,2022-09-30,981086750,1150.00,2300.00,2,135.99,Линейка 149,Бренд 3,Номенклатура 212
182719,2022-09-30,981086800,1150.00,1150.00,1,68.00,Линейка 149,Бренд 3,Номенклатура 212
182720,2022-09-30,981285250,2290.00,4580.00,2,181.13,Линейка 6,Бренд 1,Номенклатура 11


In [5]:
# По завершении месяца выгрузить эксель ozon_past_months
#ozon.to_excel('ozon_past_months.xlsx', index=False)